# Quantum Reservoir Computing (Fujii-Nakajima) で時系列2値分類

このノートブックは **Google Colab 実行**を前提に、
- GitHubからリポジトリをclone
- 可視化コードは外部モジュール (`src/spin_viz.py`) をimport
- QRC計算本体はNotebook内に記述
という構成です。


## 0. Colabセットアップ (GitHub clone + install)

`REPO_URL` を確認して実行してください。


In [ ]:
import os
import sys
import pathlib
import subprocess

REPO_URL = "https://github.com/dainfinity/qrc_classification_demo.git"
REPO_DIR = "qrc_classification_demo"

if not os.path.exists(REPO_DIR):
    subprocess.run(["git", "clone", REPO_URL, REPO_DIR], check=True)

os.chdir(REPO_DIR)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements-colab.txt"], check=True)

repo_root = pathlib.Path.cwd()
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

print("Working directory:", repo_root)


## 1. データ読み込み

固定長系列 `L` の2値分類データをCSVから読み込みます。


In [ ]:
import os
from itertools import repeat
from concurrent.futures import ProcessPoolExecutor

import numpy as np
import pandas as pd
from scipy.linalg import expm
import ipywidgets as widgets
from IPython.display import display

SEED = 1234
np.random.seed(SEED)

USE_PARALLEL = True
N_WORKERS = min(4, os.cpu_count() or 2)
print(f"USE_PARALLEL={USE_PARALLEL}, N_WORKERS={N_WORKERS}, SEED={SEED}")

train_X_raw = pd.read_csv("data/train_X.csv").values.astype(np.float64)
train_y = pd.read_csv("data/train_y.csv")["label"].values.astype(int)
test_X_raw = pd.read_csv("data/test_X.csv").values.astype(np.float64)
test_y = pd.read_csv("data/test_y.csv")["label"].values.astype(int)

M_train, L = train_X_raw.shape
M_test, L2 = test_X_raw.shape
assert L == L2
print(f"Train: {train_X_raw.shape}, Test: {test_X_raw.shape}, sequence length L={L}")


## 2. 多体系スピン可視化とハイパーパラメータ

`src/spin_viz.py` のウィジェットで以下を設定します。
- トポロジー (全結合 / 1D鎖 / 2D近接)
- スピン数 `N`
- 結合幅 `J width`
- 横磁場 `h field`

Fujii-Nakajimaスキームでは、自由パラメータは主に `J width`, `h field`, `tau(時間発展長)` です。


In [ ]:
from src.spin_viz import build_spin_control_panel

panel, controls = build_spin_control_panel(default_n=6, seed=SEED)
display(panel)

tau_slider = widgets.FloatSlider(value=0.20, min=0.02, max=1.00, step=0.02, description="tau")
display(tau_slider)

print("計算前に上のパラメータを調整してください。")


## 3. Fujii-Nakajima スキームの数式

入力は各時刻で必ず `[0,1]` にスケーリングしてから注入します。

1. 入力スケーリング
\[
	ilde{u}_t = \mathrm{clip}\left(rac{u_t-u_{\min}}{u_{\max}-u_{\min}},\,0,\,1ight)
\]

2. Ry入力注入（入力サイトを毎時刻初期化）
\[
|\psi_{\mathrm{in}}(	ilde{u}_t)angle = R_y\!\left(2rcsin\sqrt{	ilde{u}_t}ight)|0angle
= \sqrt{1-	ilde{u}_t}|0angle + \sqrt{	ilde{u}_t}|1angle
\]

3. 合成状態を作って時間発展
\[
ho_t^{\mathrm{in}} = |\psi_{\mathrm{in}}angle\langle\psi_{\mathrm{in}}|,\quad
ho_t^{\mathrm{joint}} = ho_t^{\mathrm{in}} \otimes ho_{t-1}^{\mathrm{res}}
\]
\[
U = e^{-iH	au},\quad
ho_t' = Uho_t^{\mathrm{joint}}U^\dagger
\]

4. 観測とリセット
\[
r_t^{(i)} = \mathrm{Tr}(ho_t' Z_i),\quad i=0,\dots,N-1
\]
\[
ho_t^{\mathrm{res}} = \mathrm{Tr}_{\mathrm{in}}(ho_t')
\]

ここで
\[
H = \sum_{(i,j)\in E}J_{ij}Z_iZ_j + h\sum_{i=0}^{N-1}X_i
\]
で、`E` は選択トポロジーで決まります。


In [ ]:
SIGMA_X = np.array([[0, 1], [1, 0]], dtype=np.complex128)
SIGMA_Z = np.array([[1, 0], [0, -1]], dtype=np.complex128)
IDENTITY = np.eye(2, dtype=np.complex128)


def kron_many(ops):
    out = np.array([[1.0 + 0.0j]])
    for op in ops:
        out = np.kron(out, op)
    return out


def site_operator(n_spins, site, op):
    mats = [IDENTITY] * n_spins
    mats[site] = op
    return kron_many(mats)


def zz_operator(n_spins, i, j):
    mats = [IDENTITY] * n_spins
    mats[i] = SIGMA_Z
    mats[j] = SIGMA_Z
    return kron_many(mats)


def grid_shape(n_spins):
    rows = int(np.floor(np.sqrt(n_spins)))
    while rows > 1 and n_spins % rows != 0:
        rows -= 1
    cols = int(np.ceil(n_spins / rows))
    return rows, cols


def build_edges(n_spins, topology):
    if topology == "all_to_all":
        return [(i, j) for i in range(n_spins) for j in range(i + 1, n_spins)]
    if topology == "chain_1d":
        return [(i, i + 1) for i in range(n_spins - 1)]
    if topology == "grid_2d":
        rows, cols = grid_shape(n_spins)
        edges = []
        for idx in range(n_spins):
            r, c = divmod(idx, cols)
            right = idx + 1
            down = idx + cols
            if c + 1 < cols and right < n_spins:
                edges.append((idx, right))
            if r + 1 < rows and down < n_spins:
                edges.append((idx, down))
        return edges
    raise ValueError(f"Unknown topology: {topology}")


def build_hamiltonian_and_observables(n_spins, topology, j_width, h_field, seed=SEED):
    x_ops = [site_operator(n_spins, i, SIGMA_X) for i in range(n_spins)]
    z_ops = [site_operator(n_spins, i, SIGMA_Z) for i in range(n_spins)]
    edges = build_edges(n_spins, topology)

    rng = np.random.default_rng(seed)
    dim = 2 ** n_spins
    h_int = np.zeros((dim, dim), dtype=np.complex128)
    for (i, j) in edges:
        jij = rng.uniform(-j_width, j_width)
        h_int += jij * zz_operator(n_spins, i, j)

    h_field_term = np.zeros((dim, dim), dtype=np.complex128)
    for i in range(n_spins):
        h_field_term += h_field * x_ops[i]

    h0 = h_int + h_field_term
    return h0, z_ops


def minmax_scale_to_unit_interval(x, u_min, u_max):
    if abs(u_max - u_min) < 1e-12:
        return np.zeros_like(x)
    z = (x - u_min) / (u_max - u_min)
    return np.clip(z, 0.0, 1.0)


def pure_input_density_from_u(u01):
    u01 = float(np.clip(u01, 0.0, 1.0))
    amp0 = np.sqrt(1.0 - u01)
    amp1 = np.sqrt(u01)
    psi = np.array([amp0, amp1], dtype=np.complex128)
    return np.outer(psi, psi.conj())


def partial_trace_input_qubit(rho_joint, n_spins):
    d_res = 2 ** (n_spins - 1)
    rho4 = rho_joint.reshape(2, d_res, 2, d_res)
    return np.trace(rho4, axis1=0, axis2=2)


def sequence_to_state_matrix_fn(sequence_raw, params):
    n_spins = params["n_spins"]
    topology = params["topology"]
    j_width = params["j_width"]
    h_field = params["h_field"]
    tau = params["tau"]
    seed = params["seed"]
    u_min = params["u_min"]
    u_max = params["u_max"]

    h0, z_ops = build_hamiltonian_and_observables(n_spins, topology, j_width, h_field, seed=seed)
    u_op = expm(-1j * h0 * tau)
    u_dag = u_op.conj().T

    d_res = 2 ** (n_spins - 1)
    rho_res = np.zeros((d_res, d_res), dtype=np.complex128)
    rho_res[0, 0] = 1.0 + 0.0j

    sequence = minmax_scale_to_unit_interval(np.asarray(sequence_raw, dtype=np.float64), u_min, u_max)
    states = np.zeros((len(sequence), n_spins), dtype=np.float64)

    for t, u_t in enumerate(sequence):
        rho_in = pure_input_density_from_u(u_t)
        rho_joint = np.kron(rho_in, rho_res)
        rho_evolved = u_op @ rho_joint @ u_dag

        for i in range(n_spins):
            states[t, i] = float(np.real(np.trace(rho_evolved @ z_ops[i])))

        rho_res = partial_trace_input_qubit(rho_evolved, n_spins)

    return states


def build_matrices_for_dataset(X_raw, params, use_parallel=True, n_workers=2):
    if use_parallel:
        try:
            with ProcessPoolExecutor(max_workers=n_workers) as ex:
                mats = list(ex.map(sequence_to_state_matrix_fn, X_raw, repeat(params)))
            return np.stack(mats, axis=0)
        except Exception as e:
            print(f"Parallel execution failed ({e}), fallback to sequential.")

    mats = [sequence_to_state_matrix_fn(seq, params) for seq in X_raw]
    return np.stack(mats, axis=0)


def predict_1nn_frobenius(train_mats, train_labels, test_mats):
    preds = []
    for test_mat in test_mats:
        diffs = train_mats - test_mat[None, :, :]
        dists = np.linalg.norm(diffs, ord="fro", axis=(1, 2))
        nn_idx = int(np.argmin(dists))
        preds.append(int(train_labels[nn_idx]))
    return np.array(preds, dtype=int)


## 4. 実行（入力スケーリング → リザバー行列化 → 1-NN）

- 入力は必ず `[0,1]` にスケーリング
- train/testそれぞれでリザバー状態行列を作成
- 行列を保存して、Frobenius距離の1-NNで評価


In [ ]:
# 入力スケーリング用の統計量は train から取得
u_min = float(train_X_raw.min())
u_max = float(train_X_raw.max())
print(f"Input scaling range from train: u_min={u_min:.4f}, u_max={u_max:.4f}")

reservoir_params = {
    "n_spins": int(controls["n_spins"].value),
    "topology": controls["topology"].value,
    "j_width": float(controls["j_width"].value),
    "h_field": float(controls["h_field"].value),
    "tau": float(tau_slider.value),
    "seed": SEED,
    "u_min": u_min,
    "u_max": u_max,
}

print("Reservoir params (Fujii-Nakajima):")
for k, v in reservoir_params.items():
    print(f"  {k}: {v}")

train_mats = build_matrices_for_dataset(
    train_X_raw, reservoir_params, use_parallel=USE_PARALLEL, n_workers=N_WORKERS
)
test_mats = build_matrices_for_dataset(
    test_X_raw, reservoir_params, use_parallel=USE_PARALLEL, n_workers=N_WORKERS
)

np.save("data/train_reservoir_matrices.npy", train_mats)
np.save("data/test_reservoir_matrices.npy", test_mats)
print("saved: data/train_reservoir_matrices.npy")
print("saved: data/test_reservoir_matrices.npy")

pred = predict_1nn_frobenius(train_mats, train_y, test_mats)
acc = (pred == test_y).mean()
print(f"1-NN accuracy (Frobenius): {acc:.3f}")


## 5. 注意

この実装は **時間多重化なし** です。特徴量は各時刻で1回観測した `N` 次元ベクトルのみを使っています。
